# Construção do grafo

In [1]:
import ast
import random
import re
import uuid
from datetime import datetime

import networkx as nx
import numpy as np
import pandas as pd
import rapidfuzz

In [2]:
scraped_jobs = pd.read_csv("merged_data.csv")
technical_skills = pd.read_csv("technical_skills_final.csv")

In [3]:
# Associando um novo UUID para cada job
scraped_jobs["id"] = scraped_jobs["id"].astype("str")
for i in range(scraped_jobs.shape[0]):
    scraped_jobs.at[i, "id"] = str(uuid.uuid4())

## Pré-processamento das localizações

In [4]:
location_to_region = {
    # Mapeamento dos estados para suas respectivas regiões
    "Acre": "North",
    "Alagoas": "Northeast",
    "Amapá": "North",
    "Amazonas": "North",
    "Bahia": "Northeast",
    "Ceará": "Northeast",
    "Distrito Federal": "Central-West",
    "Espírito Santo": "Southeast",
    "Goiás": "Central-West",
    "Maranhão": "Northeast",
    "Mato Grosso": "Central-West",
    "Mato Grosso do Sul": "Central-West",
    "Minas Gerais": "Southeast",
    "Pará": "North",
    "Paraíba": "Northeast",
    "Paraná": "South",
    "Pernambuco": "Northeast",
    "Piauí": "Northeast",
    "Rio de Janeiro": "Southeast",
    "Rio Grande do Norte": "Northeast",
    "Rio Grande do Sul": "South",
    "Rondônia": "North",
    "Roraima": "North",
    "Santa Catarina": "South",
    "São Paulo": "Southeast",
    "Sergipe": "Northeast",
    "Tocantins": "North",
    # Outras localidades disponíveis nas vagas
    "Federal District": "Central-West",
    "Belo Horizonte": "Southeast",
    "Porto Alegre": "South",
    "Curitiba": "South",
    "Campinas": "Southeast",
    "Ribeirão Preto": "Southeast",
    "Natal": "Northeast",
    "Recife": "Northeast",
    "Vitoria": "Southeast",
    "Londrina": "South",
    "Goiania": "Central-West",
    "Brasilia": "Central-West",
    "Salvador": "Northeast",
    "Florianopolis": "South",
    "Fortaleza": "Northeast",
    "Belem": "North",
    "Manaus": "North",
    "João Pessoa": "Northeast",
    "Cuiaba": "Central-West",
}


# Remapeando localidades para a região a que pertencem
def map_location(location):
    """
    Remaps a location to its corresponding region in Brazil.
    Args:
        location (str): The location string to be remapped.
    Returns:
        str: The region corresponding to the location, or the original location if not found.
    """
    for state, region in location_to_region.items():
        if state in location:
            return region
    return location


scraped_jobs["location"] = scraped_jobs["location"].apply(map_location)

In [5]:
# Imprimindo as localizações das vagas não remotas
print(
    f"Regiões para vagas não remotas: {scraped_jobs[scraped_jobs['remote'] == False]['location'].unique()}"
)

# Imprimindo as localizações das vagas remotas
print(
    f"Regiões para vagas remotas: {scraped_jobs[scraped_jobs['remote'] == True]['location'].unique()}"
)

Regiões para vagas não remotas: ['Southeast' 'South' 'Northeast' 'Central-West' 'North']
Regiões para vagas remotas: ['Brazil' 'Southeast' 'South' 'Northeast' 'Central-West' 'North'
 'Latin America']


Todas as vagas não remotas tiveram sua localização identificada. Para vagas remotas, faz sentido manter "Brasil" se não houver sido especificado a localização.

In [6]:
# Mudando "Latin America" para "Brazil" para manter a consistência
scraped_jobs["location"] = scraped_jobs["location"].replace("Latin America", "Brazil")

## Identificação das habilidades técnicas a partir do título e da descrição

In [7]:
def get_token_splitter(skill):
    """
    Returns a regex pattern to split text into tokens, ensuring characters existing in the given skill are preserved.
    Args:
        skill (str): The skill string to determine which characters to preserve.
    Returns:
        str: The regex pattern for token splitting.
    """
    # Encontrando todos os caracteres que não são letras no conhecimento técnico
    non_letters = set(re.findall(r"[^a-zA-Z]", skill))
    escaped_non_letters = "".join([re.escape(c) for c in non_letters])

    # Criando um padrão regex que corresponde a sequências de caracteres que não são letras, dígitos,
    #    $, #, +, ou qualquer um dos caracteres não-letras presentes na skill
    # Isso garante que caracteres como '+' em 'C++' sejam preservados como parte do token, e
    #    evita dividir 'C++' em 'C' e '' o que poderia levar a falsos positivos
    token_splitter = re.compile(rf"[^\w\$\#\+{escaped_non_letters}]+")
    return token_splitter

In [8]:
def remove_numbers(string):
    """
    Removes all numeric characters from the input string.
    Args:
        string (str): The input string from which numbers should be removed.
    Returns:
        str: The input string with all numeric characters removed.
    """
    return re.sub(r"\d+", "", string)

In [9]:
def extract_skills(job_title, job_description):
    """
    Extracts technical skills from job title and description. The skills are matched using exact matching for skills with less than 6 characters, and fuzzy matching (normalized Indel similarity) for skills with 6 or more characters.
    Args:
        job_title (str): The job title.
        job_description (str): The job description.
    Returns:
        list: A list of extracted technical skills.
    """
    found_technical_skills = set()

    # Removendo números (números de telefone, anos de experiência, versões de frameworks, etc.) do título e da descrição
    job_title = remove_numbers(job_title)
    job_description = remove_numbers(job_description)

    title = job_title.lower()
    description = job_description.lower()

    for skill in technical_skills["skill"]:
        skill = skill.lower()
        # Se a skill contém menos de 6 caracteres, usar a correspondência exata
        if len(skill) < 6:
            # Separando os tokens do título e da descrição
            token_splitter = get_token_splitter(skill)
            split_title = re.split(token_splitter, title)
            split_description = re.split(token_splitter, description)

            if skill in split_description or skill in split_title:
                found_technical_skills.add(skill)
        else:
            # Usando fuzzy matching (normalized Indel similarity) para skills com 6 ou mais letras
            minimum_match_score = int(
                100 * len(skill) / (len(skill) + 1)
            )  # no máximo 1 inserção/remoção

            # Não considerar correspondências se a skill for maior que o texto sendo pesquisado
            if (
                len(skill) <= len(description)
                and rapidfuzz.fuzz.partial_ratio(skill, description)
                > minimum_match_score
            ):
                found_technical_skills.add(skill)
            elif (
                len(skill) <= len(title)
                and rapidfuzz.fuzz.partial_ratio(skill, title) > minimum_match_score
            ):
                found_technical_skills.add(skill)

    return list(found_technical_skills) if found_technical_skills else []

In [10]:
map_skills = True

if map_skills:
    # Criando uma nova coluna 'found_skills' para armazenar as skills extraídas antes da construção do grafo
    scraped_jobs["found_skills"] = np.empty((scraped_jobs.shape[0], 0)).tolist()

    try:
        for i in range(scraped_jobs.shape[0]):
            # Adicionando à coluna 'found_skills' as skills extraídas do título e da descrição da vaga
            scraped_jobs.at[i, "found_skills"] = extract_skills(
                scraped_jobs["name"].iloc[i], scraped_jobs["description"].iloc[i]
            )

            if (i % 100) == 0:
                print(
                    f"\rProcessadas vagas até {i}, total feito {i / scraped_jobs.shape[0]}".ljust(
                        80
                    ),
                    end="",
                )

        print(f"\rTodas as vagas processadas, total feito 100%".ljust(80))
    except KeyboardInterrupt:
        print("\nProcesso interrompido.".ljust(80))

    # Salvando o dataframe com a nova coluna em um novo arquivo CSV
    scraped_jobs.to_csv("scraped_jobs_with_skills.csv", index=False)

Todas as vagas processadas, total feito 100%                                   


In [11]:
# Salvando 50 amostras aleatórias do dataframe em um novo arquivo CSV para verificação manual
scraped_jobs.sample(n=50, random_state=random.seed(datetime.now().timestamp())).to_csv(
    "scraped_jobs_with_skills_sample.csv", index=False
)

In [12]:
# Removendo de scraped_jobs as linhas onde found_skills está vazio
# Mesmo que palavras-chave relacionadas tenham sido usadas no processo de scraping,
#    algumas vagas retornadas pela API são completamente não relacionadas à tecnologia
scraped_jobs = scraped_jobs[scraped_jobs["found_skills"].map(len) > 0]

## Construção do Grafo

In [13]:
read_from_csv = False

if read_from_csv:
    scraped_jobs = pd.read_csv("scraped_jobs_with_skills.csv")
    scraped_jobs["found_skills"] = scraped_jobs["found_skills"].apply(ast.literal_eval)

G = nx.Graph()

# Criando arestas entre as vagas e skills
for _, row in scraped_jobs.iterrows():
    # Criando um novo nó para esta vaga
    G.add_node(row["id"], type="job", location=row["location"], remote=row["remote"])
    for skill in row["found_skills"]:
        # Criando um novo nó para a skill se ela não existir
        if not G.has_node(skill):
            G.add_node(skill, type="skill")
        # Criando uma aresta entre a vaga e a skill
        G.add_edge(row["id"], skill)

# Salvando como GEXF
nx.write_gexf(G, "job_skill_graph.gexf")